In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import pickle
import time

In [2]:
# read metadata
path = "/media/SSD2/Dataset/CheXpert/CheXpert-v1.0"


df = pd.read_csv(os.path.join(path, "train.csv"))

In [3]:
df["Patient"] = df["Path"].apply(lambda x: x[26:38])
df

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,Patient
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,t00001/study
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,t00002/study
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,t00002/study
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,t00002/study
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,t00003/study
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study2/view1_...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN,t64537/study
223410,CheXpert-v1.0/train/patient64537/study1/view1_...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1.0,NaN,-1.0,NaN,NaN,NaN,t64537/study
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,Female,0,Frontal,AP,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t64538/study
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0,t64539/study


In [17]:
localpath = "/home/kavindya/data/Model/FairMedFM"
df_race = pd.read_excel(os.path.join(localpath, "CHEXPERT DEMO.xlsx"), sheet_name="CheXpert Demo 11-13")
print(df_race.iloc[1])
df = df.merge(df_race, how="left", left_on="Patient", right_on="PATIENT")
df["Race"] = df["PRIMARY_RACE"]
df = df.loc[df["PATIENT"].notna() & df["Patient"].notna()].drop(
    columns=["Patient", "PATIENT", "ETHNICITY", "AGE_AT_CXR", "PRIMARY_RACE"]
)

df

PATIENT            patient48289
GENDER                   Female
AGE_AT_CXR                   39
PRIMARY_RACE              Other
ETHNICITY       Hispanic/Latino
Name: 1, dtype: object


KeyError: 'Patient'

In [6]:
df = df[df["Sex"].isin(df["GENDER"])].drop(columns="GENDER")
df

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,Race


In [7]:
# remove age/sex/race == null
df = df[~df["Sex"].isnull()]
df = df[~df["Age"].isnull()]
df = df[~df["Race"].isnull()]
df

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,Race


In [8]:
df["Race"].value_counts()

Series([], Name: count, dtype: int64)

In [9]:
mask = df["Race"].str.contains("White", na=False)
df.loc[mask, "Race"] = "White"
df.loc[~mask, "Race"] = "Non-White"

df["No Finding"].replace(-1, 0, inplace=True)
df["No Finding"].fillna(0, inplace=True)

In [10]:
df["Race"].value_counts()

Series([], Name: count, dtype: int64)

In [11]:
df["Sex"].value_counts()

Series([], Name: count, dtype: int64)

In [12]:
df["No Finding"].value_counts()

Series([], Name: count, dtype: int64)

In [14]:
df.to_csv(os.path.join(localpath, "test.csv"), index=False)

In [15]:
# sex

tem_train = pd.read_csv("/home/yesindeed/Documents/Dropbox/CXP-split/new_train.csv")
tem_test = pd.read_csv("/home/yesindeed/Documents/Dropbox/CXP-split/new_test.csv")

df = pd.read_csv(os.path.join(path, "all.csv"))
df_train = df.loc[df["Path"].isin(tem_train["Path"])].reset_index(drop=True)
df_test = df.loc[df["Path"].isin(tem_test["Path"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

df_train.to_csv(os.path.join(path, "train.csv"), index=False)
df_test.to_csv(os.path.join(path, "test.csv"), index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/yesindeed/Documents/Dropbox/CXP-split/new_train.csv'

In [13]:
# age

tem_test = pd.read_csv("/home/yesindeed/Documents/Dropbox/cxp_test_age.csv")

df = pd.read_csv(os.path.join(path, "all.csv"))
df_test = df.loc[df["Path"].isin(tem_test["Path"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

print(len(df_test))
df_test.to_csv(os.path.join(path, "test_age.csv"), index=False)

tem_test["Age_binary"].value_counts()

33757


Age_binary
0    17081
1    17081
Name: count, dtype: int64

In [10]:
df_test["Age"].value_counts()

Age
90    1128
57     797
58     796
61     753
56     750
      ... 
20     199
21     194
26     187
19     182
18      99
Name: count, Length: 73, dtype: int64

In [6]:
# race

tem_test = pd.read_csv("/home/yesindeed/Documents/Dropbox/cxp_test_race.csv")

df = pd.read_csv(os.path.join(path, "all.csv"))
df_test = df.loc[df["Path"].isin(tem_test["Path"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

df_test.to_csv(os.path.join(path, "test_race.csv"), index=False)